In [ ]:
from models import WorkersOrm, ResumesOrm, Workload
from schemas import ResumesDTO, ResumesRelDTO, WorkersDTO, WorkersRelDTO
from database import session_factory, sync_engine

from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload

Без relationship

In [ ]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .limit(2))
    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm}")
    result_dto = [WorkersDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto}")

C relationship

In [5]:
with session_factory() as session:
query = (
    select(WorkersOrm)
    .options(selectinload(WorkersOrm.resumes))
    .limit(2))
res = session.execute(query)
result_orm = res.scalars().all()
# print(f"{result_orm}")
result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm]
print(f"{result_dto}")

2024-02-04 23:12:36,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-04 23:12:36,923 INFO sqlalchemy.engine.Engine SELECT workers.id, workers.username 
FROM workers 
 LIMIT %(param_1)s::INTEGER
2024-02-04 23:12:36,926 INFO sqlalchemy.engine.Engine [cached since 23.88s ago] {'param_1': 2}
2024-02-04 23:12:36,943 INFO sqlalchemy.engine.Engine SELECT resumes.worker_id AS resumes_worker_id, resumes.id AS resumes_id, resumes.title AS resumes_title, resumes.compensation AS resumes_compensation, resumes.workload AS resumes_workload, resumes.created_at AS resumes_created_at, resumes.updated_at AS resumes_updated_at 
FROM resumes 
WHERE resumes.worker_id IN (%(primary_keys_1)s::INTEGER, %(primary_keys_2)s::INTEGER)
2024-02-04 23:12:36,949 INFO sqlalchemy.engine.Engine [cached since 23.86s ago] {'primary_keys_1': 1, 'primary_keys_2': 2}
[WorkersRelDTO(username='Jack', id=1, resumes=[ResumesDTO(title='Python Junior Developer', compensation=50000, workload=<Workload.fulltime: 'fulltime'>

JOIN

In [ ]:
with session_factory() as session:
    query = (
        select(
            ResumesOrm.workload,
            func.avg(ResumesOrm.compensation)
            .cast(Integer)
            .label("avg_compensation"),
        )
        .select_from(ResumesOrm)
        .filter(
            and_(
                ResumesOrm.title.contains('Python'),
                ResumesOrm.compensation > 40000,
            )
        )
        .group_by(ResumesOrm.workload)
        .having(func.avg(ResumesOrm.compensation) > 70000)
    )
    print(query.compile(compile_kwargs={"literal_binds": True}))
    res = session.execute(query)
    result = res.all()
    print(f"{result=}")

